In [53]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from chromedriver_py import binary_path
from selenium.webdriver.support.ui import Select
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException


your_url = 'https://statis.msmt.cz/statistikyvs/prubeh.aspx'
cService = webdriver.ChromeService(executable_path=binary_path)
browser = webdriver.Chrome(service=cService)
browser.get(your_url)

dfs = []

def get_options(xpath, start_from=1):  
    options = [None for _ in range(start_from+1)]
    # try:
    index = start_from
    while index < len(options):
        select = Select(browser.find_element(By.XPATH, xpath))
        options = select.options
        text = select.options[index].text
        select.select_by_index(index) 
        yield index, text
        index += 1

    # except StaleElementReferenceException:
    #     print('StaleElementReferenceException')
        
# Iterate over options of dropdown menus  
print("select1")
for i, select1_text in get_options('/html/body/form/div[3]/table/tbody/tr[2]/td[2]/select', start_from=16):  
    print("select2")
    for j, select2_text in get_options('/html/body/form/div[3]/table/tbody/tr[3]/td[2]/select'):  
        print("select3")
        for k, select3_text in get_options('/html/body/form/div[3]/table/tbody/tr[4]/td[2]/select'):  
            print("select4")
            for l, select4_text in get_options('/html/body/form/div[3]/table/tbody/tr[5]/td[2]/select'):  
                print(i, select1_text, j, k, l)
                try:
                    table = browser.find_element(By.XPATH, '/html/body/form/div[3]/table/tbody/tr[7]/td/div/table')
                    rows = table.find_elements(By.TAG_NAME, 'tr')

                    data = []
                    for i, row in enumerate(rows): 
                        if i == 0:  
                            cols = row.find_elements(By.TAG_NAME, 'th')  
                        else:  
                            cols = row.find_elements(By.TAG_NAME, 'td')
                        cols = [ele.text for ele in cols]
                        data.append([ele for ele in cols if ele])

                    df = pd.DataFrame(data[1:], columns=data[0]) 
                    df["vysoká škola"] = select1_text
                    df["fakulta"] = select2_text
                    df["studijní program"] = select3_text
                    df["rok zápisu"] = select4_text
                    dfs.append(df)
                except NoSuchElementException:
                    print('No table found')


select1
select2
select3
select4
16 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 1
21 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 2
20 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 3
19 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 4
18 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 5
17 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 6
16 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 7
15 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 8
14 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 9
13 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 10
12 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 11
11 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 12
10 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 13
9 27000 Vysoká škola báňská - Technická univerzita Ostrava 1 1 14
8 27000 Vysoká škola báňská - Technická 

In [54]:
df = pd.concat(dfs)

In [55]:
df.to_csv('./data/scraped_data_part2.csv', index=False)